<h3>Database-style DataFrame Merges</h3>

In [1]:
import pandas as pd
import numpy as np

In [4]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})

In [5]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [6]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


In [7]:
#many-to-one merge situation
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


<p>Note that I didn’t specify which column to join on. If not specified, merge uses the
overlapping column names as the keys. It’s a good practice to specify explicitly, though:</p>

In [8]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [10]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})

In [11]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [12]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


<p>You probably noticed that the 'c' and 'd' values and associated data are missing from
the result. By default merge does an 'inner' join; the keys in the result are the intersection.
Other possible options are 'left', 'right', and 'outer'. The outer join takes the
union of the keys, combining the effect of applying both left and right joins</p>

In [13]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


In [15]:
#Many-to-many merges have well-defined though not necessarily intuitive behavior.
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [16]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


<p>Many-to-many joins form the Cartesian product of the rows. Since there were 3 'b'
rows in the left DataFrame and 2 in the right one, there are 6 'b' rows in the result.
The join method only affects the distinct key values appearing in the result:</p>

In [17]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


<p>To merge with multiple keys, pass a list of column names:</p>

In [18]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],'key2': ['one', 'two', 'one'],'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],'key2': ['one', 'one', 'one', 'two'],'rval': [4, 5, 6, 7]})

In [19]:
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [20]:
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [21]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


To determine which key combinations will appear in the result depending on the choice
of merge method, think of the multiple keys as forming an array of tuples to be used
as a single join key (even though it’s not actually implemented that way).<br>
A last issue to consider in merge operations is the treatment of overlapping column
names. While you can address the overlap manually (see the later section on renaming
axis labels), merge has a suffixes option for specifying strings to append to overlapping
names in the left and right DataFrame objects:

In [22]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [23]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


<h3>Merging on Index</h3>

In [4]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
print(left1)
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5
   group_val
a        3.5
b        7.0


In [5]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [7]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [9]:
#With hierarchically-indexed data, things are a bit more complicated:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],'key2': [2000, 2001, 2002, 2001, 2002],'data': np.arange(5.)})
print(lefth)
righth = pd.DataFrame(np.arange(12).reshape((6, 2)), index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],[2001, 2000, 2000, 2000, 2001, 2002]], columns=['event1', 'event2'])
print(righth)

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0
             event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11


In [10]:
#In this case, you have to indicate multiple columns to merge on as a list (pay attention to the handling of duplicate index values):
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [11]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [13]:
#Using the indexes of both sides of the merge is also not an issue
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'], columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]], index=['b', 'c', 'd', 'e'], columns=['Missouri', 'Alabama'])
print(left2)
print(right2)

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0
   Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0


In [14]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


DataFrame has a more convenient join instance for merging by index. It can also be
used to combine together many DataFrame objects having the same or similar indexes
but non-overlapping columns. In the prior example, we could have written:

In [15]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


In part for legacy reasons (much earlier versions of pandas), DataFrame’s join method
performs a left join on the join keys. It also supports joining the index of the passed
DataFrame on one of the columns of the calling DataFrame:

In [16]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Lastly, for simple index-on-index merges, you can pass a list of DataFrames to join as
an alternative to using the more general concat function described below:

In [17]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]], index=['a', 'c', 'e', 'f'], columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [18]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [19]:
left2.join([right2, another], how='outer')

C:\Users\mramirez\Anaconda3\lib\site-packages\pandas\core\frame.py:6848: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  verify_integrity=True)


,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
b,NaN,NaN,7.0,8.0,NaN,NaN
c,3.0,4.0,9.0,10.0,9.0,10.0
d,NaN,NaN,11.0,12.0,NaN,NaN
e,5.0,6.0,13.0,14.0,11.0,12.0
f,NaN,NaN,NaN,NaN,16.0,17.0


<h3>Concatenating Along an Axis</h3>

In [2]:
arr = np.arange(12).reshape((3,4))

In [3]:
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [4]:
np.concatenate([arr,arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [5]:
s1 = pd.Series([0,1], index=['a','b'])
s2 = pd.Series([2,3,4], index=['c','d','e'])
s3 = pd.Series([5,6], index=['f','g'])

In [6]:
#Calling concat with these object in a list glues together the values and indexes:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

By default concat works along axis=0 , producing another Series. If you pass axis=1 , the
result will instead be a DataFrame ( axis=1 is the columns):

In [7]:
pd.concat([s1,s2,s3], axis=1)

C:\Users\mramirez\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [8]:
s4 = pd.concat([s1*5,s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [9]:
pd.concat([s1,s4], axis=1)

C:\Users\mramirez\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [10]:
pd.concat([s1,s4], axis=1, join='inner')

,0,1
a,0,0
b,1,5


In [11]:
pd.concat([s1,s4], axis=1,join_axes=[['a','c','b','e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


One issue is that the concatenated pieces are not identifiable in the result. Suppose
instead you wanted to create a hierarchical index on the concatenation axis. To do this,
use the keys argument:

In [12]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [13]:
# Much more on the unstack function later
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [14]:
#In the case of combining Series along axis=1 , the keys become the DataFrame column headers:
pd.concat([s1,s2,s3], axis=1, keys=['one', 'two', 'three'])

C:\Users\mramirez\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


The same logic extends to DataFrame objects:

In [15]:
df1 = pd.DataFrame(np.arange(6).reshape(3,2), index=['a','b','c'], columns=['one', 'two'])
df2 = pd.DataFrame(5+np.arange(4).reshape(2,2), index=['a','c'], columns=['three', 'four'])
print(df1)
print(df2)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


In [16]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

C:\Users\mramirez\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

There are a couple of additional arguments governing how the hierarchical index is
created

In [17]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], names=['upper', 'lower'])

C:\Users\mramirez\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

In [18]:
#A last consideration concerns DataFrames in which the row index is not meaningful in the context of the analysis:
df1 = pd.DataFrame(np.random.randn(3,4), columns=['a','b','c','d'])
df2 = pd.DataFrame(np.random.randn(2,3), columns=['b','d','a'])
print(df1)
print(df2)

          a         b         c         d
0  0.056312 -0.113858  0.494099 -0.453329
1 -0.294557  0.735420 -0.330738 -0.710578
2  0.637853  0.880588 -0.554396  0.730148
          b         d         a
0 -0.192799  0.876186  0.014955
1  0.420949 -0.864044  1.707380


In [19]:
#In this case, you can pass ignore_index=True :
pd.concat([df1,df2], ignore_index=True)

C:\Users\mramirez\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,a,b,c,d
0,0.056312,-0.113858,0.494099,-0.453329
1,-0.294557,0.735420,-0.330738,-0.710578
2,0.637853,0.880588,-0.554396,0.730148
3,0.014955,-0.192799,NaN,0.876186
4,1.707380,0.420949,NaN,-0.864044


# Combining Data with Overlap
You may have two datasets whose indexes overlap in full or part. As a motivating example, consider NumPy’s where function, which expressed a vectorized if-else:

In [20]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f','e','d','c','b','a'])
b = pd.Series(np.arange(len(a), dtype=np.float64), index=['f','e','d','c','b','a'])

In [21]:
b[-1]=np.nan

In [22]:
print(a)
print(b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64
f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64


In [23]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [24]:
#Series has a combine_first method, which performs the equivalent of this operation plus data alignment:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

With DataFrames, combine_first naturally does the same thing column by column, so
you can think of it as “patching” missing data in the calling object with data from the
object you pass:

In [25]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],'b': [np.nan, 2., np.nan, 6.], 'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.], 'b': [np.nan, 3., 4., 6., 8.]})

In [26]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


<h2>Reshaping and Pivoting</h2>
• stack : this “rotates” or pivots from the columns in the data to the rows<br>
• unstack : this pivots from the rows into the columns

In [27]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),index=pd.Index(['Ohio', 'Colorado'], name='state'), columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [28]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [29]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


By default the innermost level is unstacked (same with stack ). You can unstack a dif-
ferent level by passing a level number or name:

In [30]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [31]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


Unstacking might introduce missing data if all of the values in the level aren’t found in
each of the subgroups:

In [33]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [34]:
data2 = pd.concat([s1,s2], keys=['one','two'])

In [35]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [36]:
#Stacking filters out missing data by default, so the operation is easily invertible:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [37]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [38]:
#When unstacking in a DataFrame, the level unstacked becomes the lowest level in the result:
df = pd.DataFrame({'left': result, 'right': result + 5},columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [39]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [40]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

In [41]:
data = pd.DataFrame({'k1': ['one']*3 + ['two']*4,
                    'k2':[1,1,2,3,3,4,4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


In [42]:
#duplicated returns a boolean Series indicating whether each row is a duplicate or not:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

In [43]:
#drop_duplicates returns a DataFrame where the duplicated array is True :
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


Suppose we had an additional column of values and wanted to filter duplicates only based on the 'k1' column:

In [44]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [45]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


duplicated and drop_duplicates by default keep the first observed value combination. Passing keep='last' will return the last one:

In [46]:
data.drop_duplicates(['k1','k2'], keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


<h3>Transforming Data Using a Function or Mapping</h3>

In [47]:
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami','corned beef', 'Bacon', 'pastrami', 'honey ham','nova lox'],'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [48]:
#add a column indicating the type of animal that each food came from.
meat_to_animal = {
'bacon': 'pig',
'pulled pork': 'pig',
'pastrami': 'cow',
'corned beef': 'cow',
'honey ham': 'pig',
'nova lox': 'salmon'
}

The map method on a Series accepts a function or dict-like object containing a mapping,
but here we have a small problem in that some of the meats above are capitalized and
others are not. Thus, we also need to convert each value to lower case:

In [49]:
data['animal'] = data['food'].map(str.lower).map(meat_to_animal)
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


In [50]:
#We could also have passed a function that does all the work:
data['food'].map(lambda x: meat_to_animal[x.lower()])

0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

<h3>Replacing Values</h3>

In [52]:
data = pd.Series([1., -999., 2., -999., -1000., 3.])

In [53]:
data

0       1.0
1    -999.0
2       2.0
3    -999.0
4   -1000.0
5       3.0
dtype: float64

The -999 values might be sentinel values for missing data. To replace these with NA
values that pandas understands, we can use replace, producing a new Series:

In [54]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [55]:
#If you want to replace multiple values at once, you instead pass a list then the substitute value:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [56]:
#To use a different replacement for each value, pass a list of substitutes:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [57]:
#The argument passed can also be a dict:
data.replace({-999: np.nan, -1000:0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

<h3>Renaming Axis Indexes</h3>

In [3]:
data = pd.DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'],columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [4]:
#Like a Series, the axis indexes have a map method:
data.index.map(str.upper)

Index(['OHIO', 'COLORADO', 'NEW YORK'], dtype='object')

In [5]:
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [7]:
#If you want to create a transformed version of a data set without modifying the original, a useful method is rename:
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [9]:
#Notably, rename can be used in conjunction with a dict-like object providing new values for a subset of the axis labels:
data.rename(index={'OHIO': 'INDIANA'}, columns={'three':'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


rename saves having to copy the DataFrame manually and assign to its index and col
umns attributes. Should you wish to modify a data set in place, pass inplace=True:

In [10]:
_ = data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


<h3>Discretization and Binning</h3>

In [2]:
#group them into discrete age buckets:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]

In [3]:
bins = [18, 25, 35, 60, 100]

In [4]:
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

The object pandas returns is a special Categorical object. You can treat it like an array
of strings indicating the bin name; internally it contains a levels array indicating the
distinct category names along with a labeling for the ages data in the labels attribute:

In [10]:
cats.describe()

,counts,freqs
categories,,
"(18, 25]",5,0.416667
"(25, 35]",3,0.250000
"(35, 60]",3,0.250000
"(60, 100]",1,0.083333


In [11]:
#levels is changed for categories
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [13]:
pd.value_counts(cats)

(18, 25]     5
(35, 60]     3
(25, 35]     3
(60, 100]    1
dtype: int64

Consistent with mathematical notation for intervals, a parenthesis means that the side
is open while the square bracket means it is closed (inclusive). Which side is closed can
be changed by passing right=False :

In [14]:
pd.cut(ages, [18, 26, 36, 61, 100], right=False)

[[18, 26), [18, 26), [18, 26), [26, 36), [18, 26), ..., [26, 36), [61, 100), [36, 61), [36, 61), [26, 36)]
Length: 12
Categories (4, interval[int64]): [[18, 26) < [26, 36) < [36, 61) < [61, 100)]

In [15]:
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']

In [17]:
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

In [18]:
data = np.random.rand(20)

In [19]:
pd.cut(data, 4, precision=2)

[(0.07, 0.3], (0.3, 0.53], (0.53, 0.77], (0.77, 1.0], (0.77, 1.0], ..., (0.77, 1.0], (0.3, 0.53], (0.07, 0.3], (0.3, 0.53], (0.53, 0.77]]
Length: 20
Categories (4, interval[float64]): [(0.07, 0.3] < (0.3, 0.53] < (0.53, 0.77] < (0.77, 1.0]]

A closely related function, qcut , bins the data based on sample quantiles. Depending
on the distribution of the data, using cut will not usually result in each bin having the
same number of data points. Since qcut uses sample quantiles instead, by definition
you will obtain roughly equal-size bins:

In [20]:
data = np.random.randn(1000) #normal distributed

In [21]:
cats = pd.qcut(data, 4) #cut into quartiles
cats

[(-0.0108, 0.673], (-3.2729999999999997, -0.7], (-0.0108, 0.673], (-0.7, -0.0108], (-0.0108, 0.673], ..., (-0.0108, 0.673], (-0.0108, 0.673], (-0.7, -0.0108], (0.673, 2.644], (-0.7, -0.0108]]
Length: 1000
Categories (4, interval[float64]): [(-3.2729999999999997, -0.7] < (-0.7, -0.0108] < (-0.0108, 0.673] < (0.673, 2.644]]

In [24]:
pd.value_counts(cats)

(0.673, 2.644]                 250
(-0.0108, 0.673]               250
(-0.7, -0.0108]                250
(-3.2729999999999997, -0.7]    250
dtype: int64

Similar to cut you can pass your own quantiles (numbers between 0 and 1, inclusive):

In [25]:
pd.qcut(data, [0, 0.1, 0.5, 0.9, 1.])

[(-0.0108, 1.314], (-1.334, -0.0108], (-0.0108, 1.314], (-1.334, -0.0108], (-0.0108, 1.314], ..., (-0.0108, 1.314], (-0.0108, 1.314], (-1.334, -0.0108], (-0.0108, 1.314], (-1.334, -0.0108]]
Length: 1000
Categories (4, interval[float64]): [(-3.2729999999999997, -1.334] < (-1.334, -0.0108] < (-0.0108, 1.314] < (1.314, 2.644]]

<h3>Detecting and Filtering Outliers</h3>

In [2]:
np.random.randn(12345)

array([-0.32947806, -1.50715818, -0.82125959, ...,  0.33380197,
       -1.376982  , -0.83346307])

In [3]:
data = pd.DataFrame(np.random.randn(1000,4))

In [4]:
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.028063,-0.019235,-0.049316,-0.001927
std,0.980530,1.016165,0.985426,0.980182
min,-3.114849,-2.898574,-2.999048,-3.320386
25%,-0.669742,-0.677680,-0.717927,-0.617802
50%,0.004889,-0.009958,-0.046168,-0.008089
75%,0.606555,0.655978,0.633380,0.652800
max,2.835803,2.919582,3.329123,2.837654


Suppose you wanted to find values in one of the columns exceeding three in magnitude:

In [5]:
col = data[3]

In [6]:
col[np.abs(col) > 3]

219   -3.320386
377   -3.128841
Name: 3, dtype: float64

To select all rows having a value exceeding 3 or -3, you can use the any method on a
boolean DataFrame:

In [7]:
data[(np.abs(data)>3).any(1)]

,0,1,2,3
79,-3.109458,-0.770577,2.049711,-1.744884
219,-0.295145,0.596917,-1.682558,-3.320386
252,-3.037166,-0.907129,-1.052315,-1.044801
368,-1.199012,-1.657954,3.329123,-0.267319
377,-0.048773,0.259397,0.020350,-3.128841
831,-3.114849,1.493887,0.639195,0.691746


Values can just as easily be set based on these criteria. Here is code to cap values outside
the interval -3 to 3:

In [8]:
#The ufunc np.sign returns an array of 1 and -1 depending on the sign of the values
data[np.abs(data)>3] = np.sign(data)*3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.027802,-0.019235,-0.049645,-0.001477
std,0.979722,1.016165,0.984351,0.978744
min,-3.000000,-2.898574,-2.999048,-3.000000
25%,-0.669742,-0.677680,-0.717927,-0.617802
50%,0.004889,-0.009958,-0.046168,-0.008089
75%,0.606555,0.655978,0.633380,0.652800
max,2.835803,2.919582,3.000000,2.837654


<h3>Permutation and Random Sampling</h3>

In [9]:
df = pd.DataFrame(np.arange(5*4).reshape(5,4))

In [10]:
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [11]:
sampler = np.random.permutation(5)

In [12]:
sampler

array([3, 1, 2, 4, 0])

In [13]:
df.take(sampler)

,0,1,2,3
3,12,13,14,15
1,4,5,6,7
2,8,9,10,11
4,16,17,18,19
0,0,1,2,3


In [14]:
df.take(np.random.permutation(len(df))[:3])

,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


To generate a sample with replacement, the fastest way is to use np.random.randint to
draw random integers:

In [17]:
bag = np.array([5, 7, -1, 6, 4])
sampler = np.random.randint(0, len(bag), size=10)
print(sampler)
print(bag)

[0 4 0 2 0 3 0 3 0 0]
[ 5  7 -1  6  4]


In [18]:
draws = bag.take(sampler)
draws

array([ 5,  4,  5, -1,  5,  6,  5,  6,  5,  5])